In [1]:
import pandas

reader = lambda url, **multiargs : pandas.read_table(url, sep=",", **multiargs)

url = "/content/movies.csv"

In [2]:
origin = reader(url)

origin.iloc[:5]

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [3]:
origin.tail()

,movieId,title,genres
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation
9741,193609,Andrew Dice Clay: Dice Rules (1991),Comedy


In [4]:
null = origin.isnull().sum()

null.sum()

0

In [5]:
molist = []

for insect in origin["genres"]:

  split = insect.split("|")

  molist.append(" ".join(split))

origin["genres"] = molist

origin.iloc[:5]

,movieId,title,genres
0,1,Toy Story (1995),Adventure Animation Children Comedy Fantasy
1,2,Jumanji (1995),Adventure Children Fantasy
2,3,Grumpier Old Men (1995),Comedy Romance
3,4,Waiting to Exhale (1995),Comedy Drama Romance
4,5,Father of the Bride Part II (1995),Comedy


In [6]:
replacer = {"Sci-Fi":"SciFi", "Film-Noir":"Filnoir"}

for i, t in replacer.items():

  origin["genres"] = origin["genres"].str.replace(i, t)

origin.iloc[:5]

,movieId,title,genres
0,1,Toy Story (1995),Adventure Animation Children Comedy Fantasy
1,2,Jumanji (1995),Adventure Children Fantasy
2,3,Grumpier Old Men (1995),Comedy Romance
3,4,Waiting to Exhale (1995),Comedy Drama Romance
4,5,Father of the Bride Part II (1995),Comedy


In [7]:
result = []

for insect in origin["genres"].values:

  insect = insect.split(" ")

  for lst in insect:

    if lst not in result: result.append(lst)

len(result)

22

In [8]:
nolist = "(no genres listed)"

i = origin[origin["genres"] == nolist]

len(i)

34

In [9]:
origin = origin[origin["genres"] != nolist]

origin.iloc[:5]

,movieId,title,genres
0,1,Toy Story (1995),Adventure Animation Children Comedy Fantasy
1,2,Jumanji (1995),Adventure Children Fantasy
2,3,Grumpier Old Men (1995),Comedy Romance
3,4,Waiting to Exhale (1995),Comedy Drama Romance
4,5,Father of the Bride Part II (1995),Comedy


In [10]:
result = []

for insect in origin["genres"].values:

  insect = insect.split(" ")

  for lst in insect:

    if lst not in result: result.append(lst)

len(result)

19

In [11]:
result[:5]

['Adventure', 'Animation', 'Children', 'Comedy', 'Fantasy']

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.metrics.pairwise import cosine_similarity

tfidf = TfidfVectorizer(stop_words="english")

metrics = tfidf.fit_transform(origin["genres"])

metrics.shape

(9708, 19)

In [13]:
similarities = cosine_similarity(metrics)

similarities.shape

(9708, 9708)

In [14]:
len(similarities[0])

9708

In [15]:
similarities[0][:5]

array([1.        , 0.8136036 , 0.15259961, 0.13500041, 0.26738778])

In [16]:
intler = lambda i : origin.iloc[i, 1]

titint = lambda t : origin[origin["title"] == t].index.values[0]

In [17]:
t = titint("Jumanji (1995)")

t

1

In [18]:
moret = enumerate(similarities[int(t)])

molist = list(moret)

len(molist)

9708

In [19]:
molist[:5]

[(0, 0.8136035975025231), (1, 1.0), (2, 0.0), (3, 0.0), (4, 0.0)]

In [20]:
reverse = ~False

molist = sorted(molist, key=lambda i:i[1], reverse=reverse)

molist[:5]

[(1, 1.0), (53, 1.0), (109, 1.0), (767, 1.0), (1514, 1.0)]

In [21]:
molist = molist[:5]

origin[origin.index.isin([i[0] for i in molist])]

,movieId,title,genres
1,2,Jumanji (1995),Adventure Children Fantasy
53,60,"Indian in the Cupboard, The (1995)",Adventure Children Fantasy
109,126,"NeverEnding Story III, The (1994)",Adventure Children Fantasy
767,1009,Escape to Witch Mountain (1975),Adventure Children Fantasy
1514,2043,Darby O'Gill and the Little People (1959),Adventure Children Fantasy


In [22]:
limit = 10

title = "Jumanji (1995)"

def output(title, limit):

  morate = enumerate(similarities[int(titint(title))])
  molist = list(morate)

  morter = sorted(molist, key=lambda i:i[1], reverse=reverse)
  molist = morter[:limit]

  molter = filter(lambda i : i[0] != titint(title), molist)
  molist = list(molter)

  molist = [intler(molist[i][0]) for i in range(len(molist))]
  result = origin[origin["title"].isin(molist)]

  return result

output(title, limit)

,movieId,title,genres
53,60,"Indian in the Cupboard, The (1995)",Adventure Children Fantasy
109,126,"NeverEnding Story III, The (1994)",Adventure Children Fantasy
767,1009,Escape to Witch Mountain (1975),Adventure Children Fantasy
1514,2043,Darby O'Gill and the Little People (1959),Adventure Children Fantasy
1556,2093,Return to Oz (1985),Adventure Children Fantasy
1617,2161,"NeverEnding Story, The (1984)",Adventure Children Fantasy
1618,2162,"NeverEnding Story II: The Next Chapter, The (1...",Adventure Children Fantasy
1799,2399,Santa Claus: The Movie (1985),Adventure Children Fantasy
3574,4896,Harry Potter and the Sorcerer's Stone (a.k.a. ...,Adventure Children Fantasy


In [23]:
title = "Waiting to Exhale (1995)"

output(title, 15)

,movieId,title,genres
10,11,"American President, The (1995)",Comedy Drama Romance
47,52,Mighty Aphrodite (1995),Comedy Drama Romance
52,58,"Postman, The (Postino, Il) (1994)",Comedy Drama Romance
83,94,Beautiful Girls (1996),Comedy Drama Romance
165,195,Something to Talk About (1995),Comedy Drama Romance
191,224,Don Juan DeMarco (1995),Comedy Drama Romance
198,232,Eat Drink Man Woman (Yin shi nan nu) (1994),Comedy Drama Romance
243,281,Nobody's Fool (1994),Comedy Drama Romance
309,351,"Corrina, Corrina (1994)",Comedy Drama Romance
317,359,I Like It Like That (1994),Comedy Drama Romance


In [24]:
title = origin.iloc[3574:3575]

title

,movieId,title,genres
3574,4896,Harry Potter and the Sorcerer's Stone (a.k.a. ...,Adventure Children Fantasy


In [25]:
title = origin.iloc[3574, 1]

output(title, 5)

,movieId,title,genres
1,2,Jumanji (1995),Adventure Children Fantasy
53,60,"Indian in the Cupboard, The (1995)",Adventure Children Fantasy
109,126,"NeverEnding Story III, The (1994)",Adventure Children Fantasy
767,1009,Escape to Witch Mountain (1975),Adventure Children Fantasy
1514,2043,Darby O'Gill and the Little People (1959),Adventure Children Fantasy


In [26]:
option = ["Children", "Fantasy"]

option

['Children', 'Fantasy']

In [27]:
def starter(option, limit):

  result = []

  ontari = origin["genres"].unique()

  for item in ontari:

    spliter = item.split(" ")

    overlap = len(set(option) & set(spliter)) / len(spliter)

    if overlap > 0.5:

      result.append((item, overlap))

  result = sorted(result, key=lambda i:i[1], reverse=reverse)[:limit]

  result = [i[0] for i in result]

  return origin[origin["genres"].isin(result)].iloc[:limit]

starter(option, 5)

,movieId,title,genres
1,2,Jumanji (1995),Adventure Children Fantasy
53,60,"Indian in the Cupboard, The (1995)",Adventure Children Fantasy
109,126,"NeverEnding Story III, The (1994)",Adventure Children Fantasy
209,243,Gordy (1995),Children Comedy Fantasy
301,343,"Baby-Sitters Club, The (1995)",Children
